* Data : Naver sentiment movie corpus v1.0 https://github.com/e9t/nsmc

# 자연어 처리에서의 문제

** 텍스트는 알고리즘에 직접 넣을 수 없다!**

알고리즘은 고정 길이의 숫자형 데이터를 기대한다. 가변 길이(시퀀스)의 문자형 데이터는 받지 못한다. 문자 끼리는 직접 덧셈 뺄셈 곱셈 나눗셈을 할 수 없기 때문이다.

그럼 어떻게?

** key point : 텍스트를 어떻게 숫자로 바꿀 것인가 **

# 텍스트(자연어)를 숫자형 데이터로 나타내기
----

## * One hot encoding

* ex) 'a' -> [1, 0, 0], 'b' -> [0, 1, 0], 'c' -> [0, 0, 1]

## Count vectorizer

In [41]:
# example text for model training (SMS messages)
simple_train = ['잔디콘은 짱이에요', '엔드게임은 정말 짱이에요', '엄복동 짜증나']

In [42]:
# example response vector
is_positive = [1, 1, 0]

In [43]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [44]:
# learn the 'vocabulary' of the training data (occurs in-place)
vect.fit(simple_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [45]:
vect.get_feature_names()

['엄복동', '엔드게임은', '잔디콘은', '정말', '짜증나', '짱이에요']

In [46]:
# examine the fitted vocabulary
vect.get_feature_names()

['엄복동', '엔드게임은', '잔디콘은', '정말', '짜증나', '짱이에요']

In [47]:
# transform training data into a 'document-term matrix'
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

**Sparse representation**이다.

In [48]:
# convert sparse matrix to a dense matrix
simple_train_dtm.toarray()

array([[0, 0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0, 1],
       [1, 0, 0, 0, 1, 0]])

In [49]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

,엄복동,엔드게임은,잔디콘은,정말,짜증나,짱이에요
0,0,0,1,0,0,1
1,0,1,0,1,0,1
2,1,0,0,0,1,0


### 이 방식(count vectorizer)에서

- 각각의 단어들의 빈도는 feature가 된다.
- 각각의 문서들은 data point가 된다.

- 즉, 문서 데이터셋은 행이 각 문서이고 열이 각 단어인 행렬로 표현될 수 있다.

-  **벡터화(Vectorization)** : 텍스트 데이터셋(문서 또는 단어, 문장 등등)을 숫자형 feature 벡터로 나타내는 것.

* Bag of words 방식 : 단어들을 가방에 넣는다는 뜻. 즉 단어 사이의 순서는 사라지고 단순히 단어가 몇개 나왔느냐를 갖고 문서를 벡터화 하는 것

In [50]:
# check the type of the document-term matrix
type(simple_train_dtm)

scipy.sparse.csr.csr_matrix

In [51]:
# examine the sparse matrix contents
print(simple_train_dtm)

  (0, 2)	1
  (0, 5)	1
  (1, 1)	1
  (1, 3)	1
  (1, 5)	1
  (2, 0)	1
  (2, 4)	1


In [67]:
# build a model to predict desperation
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(simple_train_dtm, is_positive)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [68]:
# example text for model testing
simple_test = ["기생충은 정말 짱이에요"]

**예측**을 하기 위해서, 새로운 data point는 **학습용 데이터셋과 같은 feature들**을 같은 순서로 갖고 있어야 한다.

In [69]:
# transform testing data into a document-term matrix (using existing vocabulary)
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 0, 0, 1, 0, 1]])

In [70]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names())

,엄복동,엔드게임은,잔디콘은,정말,짜증나,짱이에요
0,0,0,0,1,0,1


In [71]:
# predict whether simple_test is desperate
knn.predict(simple_test_dtm)

array([1])

**요약**

- `vect.fit(train)` 학습용 데이터셋의 **사전을 학습한다.**
- `vect.transform(train)` **학습된 사전**을 이용해서 학습용 데이터셋의 문서-단어(document-term) 행렬을 만든다.
- `vect.transform(test)` **학습된 사전**을 이용해서 새로운(테스트용) 데이터셋의 문서-단어(document-term) 행렬을 만든다. 이 때 학습용 데이터셋에서 보지 못했던 단어들은 **무시한다**

# 좀 더 큰 데이터셋

In [105]:
# read file into pandas from the working directory
train = pd.read_csv('nsmc/ratings_train.txt', sep='\t').dropna()[['document', 'label']]
test = pd.read_csv('nsmc/ratings_test.txt', sep='\t').dropna()[['document', 'label']]

In [106]:
# examine the shape
train.shape

(149995, 2)

In [107]:
# examine the first 10 rows
train.head(10)

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,원작의 긴장감을 제대로 살려내지못했다.,0
7,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,액션이 없는데도 재미 있는 몇안되는 영화,1
9,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [108]:
# examine the class distribution
train.label.value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [109]:
# split X and y into training and testing sets
X_train = train.document
X_test = test.document
y_train = train.label
y_test = test.label
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(149995,)
(49997,)
(149995,)
(49997,)


# 데이터셋을 벡터화하기

In [110]:
# instantiate the vectorizer
vect = CountVectorizer()

In [111]:
# learn training data vocabulary, then use it to create a document-term matrix
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [112]:
# equivalently: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)

In [113]:
# examine the document-term matrix
X_train_dtm

<149995x293366 sparse matrix of type '<class 'numpy.int64'>'
	with 1074805 stored elements in Compressed Sparse Row format>

In [114]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<49997x293366 sparse matrix of type '<class 'numpy.int64'>'
	with 284188 stored elements in Compressed Sparse Row format>

# Naivce Bayes

In [116]:
# import and instantiate a logistic regression model
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()

In [117]:
# train the model using X_train_dtm
%time naive_bayes.fit(X_train_dtm, y_train)

CPU times: user 70.3 ms, sys: 17.9 ms, total: 88.1 ms
Wall time: 117 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [118]:
# make class predictions for X_test_dtm
y_pred_class = naive_bayes.predict(X_test_dtm)

In [119]:
y_pred_class

array([0, 0, 0, ..., 1, 0, 0])

In [120]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = naive_bayes.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([4.98849962e-01, 4.98849962e-01, 8.71972262e-03, ...,
       9.59138782e-01, 1.64400107e-04, 3.67424957e-01])

In [121]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [122]:
# calculate accuracy of class predictions
accuracy_score(y_test, y_pred_class)

0.8258095485729143

In [123]:
# calculate AUC
roc_auc_score(y_test, y_pred_prob)

0.9124151346160185